In [1]:
import pandas as pd
from pandas import DataFrame
from typing import List, Dict
import logging
import time
from functools import reduce
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects, RetryError
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

## All Coins List

In [52]:
def get_coins_list() -> List[dict]:
    return requests.get('https://api.coingecko.com/api/v3/coins/list').json()


coins_list = get_coins_list()
coins_list[:3]

[{'id': '01coin', 'symbol': 'zoc', 'name': '01coin'},
 {'id': '0-5x-long-algorand-token',
  'symbol': 'algohalf',
  'name': '0.5X Long Algorand Token'},
 {'id': '0-5x-long-altcoin-index-token',
  'symbol': 'althalf',
  'name': '0.5X Long Altcoin Index Token'}]

In [4]:
def get_coin_details(id: str, retries: int = 5) -> dict:
    for _ in range(retries):
        response = requests.get(f'https://api.coingecko.com/api/v3/coins/{id}')
        logging.debug(response.status_code)
        if response.status_code == 200: return response.json()
        else:
            time.sleep(1)
    return {}

In [43]:
coin_detail0 = get_coin_details(coins_list[0]["id"])
coin_detail1 = get_coin_details(coins_list[1]["id"])
coin_detail0

{'id': '01coin',
 'symbol': 'zoc',
 'name': '01coin',
 'asset_platform_id': None,
 'platforms': {'': ''},
 'block_time_in_minutes': 0,
 'hashing_algorithm': 'NeoScrypt',
 'categories': ['Masternodes'],
 'public_notice': None,
 'additional_notices': [],
 'localization': {'en': '01coin',
  'de': '01coin',
  'es': '01coin',
  'fr': '01coin',
  'it': '01coin',
  'pl': '01coin',
  'ro': '01coin',
  'hu': '01coin',
  'nl': '01coin',
  'pt': '01coin',
  'sv': '01coin',
  'vi': '01coin',
  'tr': '01coin',
  'ru': '01coin',
  'ja': '01coin',
  'zh': '01coin',
  'zh-tw': '01coin',
  'ko': '01coin',
  'ar': '01coin',
  'th': '01coin',
  'id': '01coin',
  'cs': '01coin',
  'da': '01coin',
  'el': '01coin',
  'hi': '01coin',
  'no': '01coin',
  'sk': '01coin',
  'uk': '01coin',
  'he': '01coin',
  'fi': '01coin',
  'bg': '01coin',
  'hr': '01coin',
  'lt': '01coin',
  'sl': '01coin'},
 'description': {'en': '',
  'de': '',
  'es': '',
  'fr': '',
  'it': '',
  'pl': '',
  'ro': '',
  'hu': '',
  'n

In [31]:
pd.DataFrame([get_coin_details(coins_list[50]["id"])])

,id,symbol,name,asset_platform_id,platforms,block_time_in_minutes,hashing_algorithm,categories,public_notice,additional_notices,...,community_score,liquidity_score,public_interest_score,market_data,community_data,developer_data,public_interest_stats,status_updates,last_updated,tickers
0,12866-lauder,realtoken-s-12866-lauder-st-detroit-mi,"RealT Token - 12866 Lauder St, Detroit, MI 48227",xdai,{'xdai': '0xb5dd2b6e0a0422e069e1d2cc3ed1653348...,0,None,[RealT Tokens],None,[Kindly be aware of <a href='https://www.coing...,...,9.159,1.0,0.0,"{'current_price': {'aed': 174.66, 'ars': 5477....","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 0, 'stars': 0, 'subscribers': 0, 'to...","{'alexa_rank': None, 'bing_matches': None}",[],2022-04-29T10:18:03.560Z,[{'base': '0X1698CD22278EF6E7C0DF45A8DEA72EDBE...


## Extract Dict Data Frame

In [49]:
def get_dict_info(coin_detail: dict, column_name: str) -> dict:
    try: dict_info: dict = coin_detail[f"{column_name}"]
    except: return {"id": [coin_detail["id"]]}
    dict_info["id"] = coin_detail["id"]
    final_dict: dict = {key: [value] for key, value in dict_info.items()}
    if '' in final_dict.keys(): final_dict.pop('')
    return final_dict


dict_info0 = get_dict_info(coin_detail0, "platforms")
dict_info1 = get_dict_info(coin_detail1, "platforms")

def get_dict_df(dict_info: dict) -> DataFrame:
    df: DataFrame = pd.DataFrame(dict_info)
    columns: List[str] = df.columns.to_list()
    columns.remove("id")
    ordered_columns: List[str] = ["id", *columns]
    return df[ordered_columns]


dict_df0 = get_dict_df(dict_info0)
dict_df1 = get_dict_df(dict_info1)


def concat_df(df1, df2) -> DataFrame:
    return pd.concat([df1, df2], ignore_index=True)

dict_df = concat_df(dict_df0, dict_df1)

def get_dict_df_all(coins_list: List[dict], column_name: str) -> DataFrame:
    coin_detail_list = [get_coin_details(coin["id"]) for coin in coins_list[:50]]
    dict_info_list = [get_dict_info(coin_detail, column_name) for coin_detail in coin_detail_list]
    dict_df_list = [get_dict_df(dict_info) for dict_info in dict_info_list]
    return reduce(concat_df, dict_df_list)

dict_df_all = get_dict_df_all(coins_list, "platforms")
dict_df_all.head(3)

,id,ethereum,polygon-pos,avalanche,harmony-shard-0,fantom,xdai,binance-smart-chain,solana
0,01coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0-5x-long-algorand-token,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0-5x-long-altcoin-index-token,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Extract Plataforms Data Frame

In [51]:
dict_location_all = get_dict_df_all(coins_list, "platforms")
dict_location_all.head(3)

,id,ethereum,polygon-pos,avalanche,harmony-shard-0,fantom,xdai,binance-smart-chain,solana
0,01coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0-5x-long-algorand-token,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0-5x-long-altcoin-index-token,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Extract Localization Data Frame

In [50]:
dict_location_all = get_dict_df_all(coins_list, "localization")
dict_location_all.head(3)

,id,en,de,es,fr,it,pl,ro,hu,nl,...,hi,no,sk,uk,he,fi,bg,hr,lt,sl
0,01coin,01coin,01coin,01coin,01coin,01coin,01coin,01coin,01coin,01coin,...,01coin,01coin,01coin,01coin,01coin,01coin,01coin,01coin,01coin,01coin
1,0-5x-long-algorand-token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,...,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token,0.5X Long Algorand Token
2,0-5x-long-altcoin-index-token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,...,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token,0.5X Long Altcoin Index Token
